In [1]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(psych,warn.conflicts=F)
library(lplyr,warn.conflicts=F)

In [2]:
# load results
data.offdeff <- readRDS('../predictions_2019/OffDef_ScorePoss_Laplace.rds')
data.sagarin <- readRDS('../predictions_2019/SagarinRating.rds')
data.dunkel <- readRDS('../predictions_2019/DunkelIndex.rds')

In [3]:
data.sagarin %>% 
    rename(PredSAG=Pred,ProbSAG=Prob) %>%
    inner_join(data.dunkel,by='ID') %>%
    rename(PredDUN=Pred,ProbDUN=Prob) %>% 
    inner_join(data.offdeff,by='ID') %>%
    rename(PredOFC=Pred,ProbOFC=Prob) -> df.mix

In [4]:
df.mix %>%
    mutate(Pred=PredSAG+PredDUN+PredOFC) %>%
    group_by(Pred) %>%
    summarise(n=n())

Pred,n
0,1022
1,155
2,117
3,984


In [5]:
(1022+984)/2278

[1] 0.880597

In [6]:
df.mix %>%
    mutate(Tot=PredSAG+PredDUN+PredOFC) %>%
    filter(Tot==1 | Tot==2) %>% sample_n(10)

ID,PredSAG,ProbSAG,PredDUN,ProbDUN,PredOFC,ProbOFC,Tot
2019_1279_1388,0,0.434,1,0.502,0,0.399,1
2019_1199_1266,1,0.601,1,0.707,0,0.461,2
2019_1308_1433,0,0.459,1,0.639,0,0.434,1
2019_1235_1243,1,0.562,0,0.439,1,0.615,2
2019_1416_1437,0,0.314,0,0.150,1,0.524,1
2019_1326_1429,1,0.536,0,0.475,0,0.408,1
2019_1328_1388,1,0.539,1,0.583,0,0.430,2
2019_1234_1328,0,0.430,1,0.506,0,0.482,1
2019_1278_1308,1,0.581,0,0.262,0,0.429,1
2019_1243_1280,1,0.522,1,0.535,0,0.418,2


#### Equi Mix

In [19]:
df.mix %>%
    mutate(Tot=PredSAG+PredDUN+PredOFC) %>%
    mutate(Prob=case_when(Tot==0 ~ pmin(ProbSAG,ProbDUN,ProbOFC), 
                          Tot==3 ~ pmax(ProbSAG,ProbDUN,ProbOFC),
                          Tot==1|Tot==2 ~ (ProbSAG+ProbDUN+ProbOFC)/3,
                          TRUE ~ 0.5)) %>%
    mutate(Pred=ifelse(Prob>0.5,1,0)) %>%
    select(ID,Pred) %>%
    write.csv('../predictions_2019/RatingMix_SAGDUNOFC.csv',quote=F,row.names=F)

#### Champion vs Outsider

In [ ]:
df.mix %>%
    mutate(Tot=PredOFC-PredSAG-PredDUN) %>%
    filter(Tot==)